# ANLP 2019 - Assignment 3


** 

<div class="alert alert-block alert-danger">Due: Wednesday, December 4th (before class)</div>

<div class="alert alert-block alert-info">
**NOTE**<br><br>

Please first fill in your name and id number at the top of the assignment, and **rename** the assignment file to **yourlastname-anlp-3.ipynb**<br><br>
Problems and questions are given in blue boxes like this one. All grey and white boxes must be filled by you (they either require code or a (brief!) discussion). <br><br>
Please hand in your assignment by the deadline via Moodle. In case of questions, you can contact the TAs or David via the usual channels.
</div>

<div class="alert alert-block alert-info">
In this assignment, you will implement a bigram part-of-speech (POS) tagger based on Hidden Markov Models from scratch. Using NLTK is disallowed, except for the modules explicitly listed below. For this, you will need to develop and utilize the following modules:<br>
1. Corpus reader and writer<br>
2. Training procedure, including smoothing<br>
3. Viterbi tagging, including unknown word handling <br>
4. Evaluation<br>
The task is mostly very straightforward, but each step requires careful design. Thus, we suggest you proceed in the following way.
</div>

In [15]:
from __future__ import division
from __future__ import unicode_literals
import matplotlib.pyplot as plt
from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict
from collections import namedtuple
import random
import sys
import traceback
from nltk.probability import (
    FreqDist,
    ConditionalFreqDist,
    ConditionalProbDist,
    DictionaryProbDist,
    DictionaryConditionalProbDist,
    LidstoneProbDist,
    MutableProbDist,
    MLEProbDist,
    RandomProbDist,
)
#Importing libraries
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

## Problem 1: Viterbi Algorithm [33 points]

<div class="alert alert-block alert-info">
First, implement the Viterbi algorithm for finding the optimal state (tag) sequence given the sequence of observations (words). <br><br>
In order to test your implementation, verify that you compute the correct state sequence for some examples from Eisner's ice cream model (see lecture) for which the solutions are known.<br><br>
Demonstrate that your algorithm computes the correct state sequence for ['3','1','3'] as in the lecture.<br><br>
Make sure that your algorithm is correct before proceeding to the other tasks! In order to do this, please also test your module with a longer observation sequence. 
</div>

In [16]:
def generate_sequence(states,sequence_length):
    
    all_sequences = []
    nodes = []
    
    depth = sequence_length
    
    def gen_seq_recur(states,nodes,depth):
        if depth == 0:
            
            all_sequences.append(nodes)
        else:
            for state in states:
                temp_nodes = list(nodes)
                temp_nodes.append(state)
                gen_seq_recur(states,temp_nodes,depth-1)
    
    gen_seq_recur(states,[],depth)
                
    return all_sequences

def score_sequences(sequences,initial_probs,transition_probs,emission_probs,obs):
    
    best_score = -1
    best_sequence = None
    
    sequence_scores = []
    for seq in sequences:
        total_score = 1
        total_score_breakdown = []
        first = True
        for i in range(len(seq)):
            state_score = 1
            # compute transitition probability score
            if first == True:
                state_score *= initial_probs[seq[i]]
                # reset first flag
                first = False
            else:  
                state_score *= transition_probs[seq[i] + "|" + seq[i-1]]
            # add to emission probability score
            state_score *= emission_probs[obs[i] + "|" + seq[i]]
            
            total_score_breakdown.append(state_score)
            total_score *= state_score
            
        sequence_scores.append(total_score)
        
    return sequence_scores


def initializeSequences(_obs):
    # Generate list of sequences
    
    seqLen = len(_obs)
    seqs = generate_sequence(states,seqLen)
    # Score sequences
    seq_scores = score_sequences(seqs,initial_probs,transition_probs,emission_probs,obs)
    
    return (seqLen,seqs,seq_scores)

states = ['C','H']
obs = ['1','2','3']
initial_probs = {'C': 0.2, 'H': 0.8}
transition_probs = {'C|C':0.6, 'C|H': 0.4, 'H|C':0.3, 'H|H':0.7}
emission_probs = {'1|C':0.5,'2|C':0.4,'3|C':0.1, '1|H':0.2, '2|H':0.4,'3|H':0.4}



sequence_length,sequences,sequence_scores = initializeSequences(obs)
# print results
print("Initial Distributions")
print (initial_probs)
print("\nTransition Probabilities")

# Generate list of sequences
sequence_length,sequences,sequence_scores = initializeSequences(obs)
# Display sequence scores
for i in range(len(sequences)):
    print("Sequence:%-60s Score:%0.6f" % (sequences[i],sequence_scores[i]))
    
# Display the winning score
print("\n Best Sequence")
print(sequences[sequence_scores.index(max(sequence_scores))],max(sequence_scores))

Initial Distributions
{'C': 0.2, 'H': 0.8}

Transition Probabilities
Sequence:['C', 'C', 'C']                                              Score:0.001440
Sequence:['C', 'C', 'H']                                              Score:0.002880
Sequence:['C', 'H', 'C']                                              Score:0.000480
Sequence:['C', 'H', 'H']                                              Score:0.003360
Sequence:['H', 'C', 'C']                                              Score:0.001536
Sequence:['H', 'C', 'H']                                              Score:0.003072
Sequence:['H', 'H', 'C']                                              Score:0.001792
Sequence:['H', 'H', 'H']                                              Score:0.012544

 Best Sequence
['H', 'H', 'H'] 0.012544000000000001


## Problem 2: HMM Training [33 points]

<div class="alert alert-block alert-info">
Second, learn parameters, i.e. the initial, transition, and emission probabilities, for your HMM from *annotated* data. Implement a maximum likelihood training procedure (with smoothing) for supervised learning of HMMs. We will be using the Wall Street Journal corpus (part of the Penn Treebank). It is a licensed corpus, so please do not redistribute the files. The zip archive provided on Moodle contains a training set, a test set, and an evaluation set. The training set (`wsj_tagged_train.tt`) and the evaluation set (`wsj_tagged_eval.tt`) are written in the commonly used CoNLL format. They are text files with two colums; the first column contains the words, the POS tags are in the second column, and empty lines delimit sentences. The test set (`wsj_tagged_test.t`) is a copy of the evaluation set with tags stripped. You should tag this test set using your tagger and then compare your results with the gold-standard ones in the provided tagged evaluation file. The corpus uses the Penn Treebank tagset.<br><br>
You are welcome to use any NLTK data structures from the two modules `nltk.corpus.reader` (and submodules) and `nltk.probability`. The latter includes a number of smoothing procedures, which you may want to apply to your corpus frequency counts. Take care to get NLTK to make the smoothed probability distributions sum to one. Experiment with unsmoothed distributions, Laplace add-one smoothing, and at least one further smoothing procedure.<br><br>
Note that your tagger will initially fail to produce output for sentences that contain words you haven't seen in training. If you have such a word $w$ appear at sentence position $t$, you will have $b_j(w) = 0$ for all states/tags $j$, and therefore $V_t(j) = 0$ for all $j$. Adapt your tagger by implementing the following crude approach to unknown words. Whenever you get $V_t(j) = 0$ for all $j$ because of an unknown word at position $t$, pretend that $b_j(w) = 1$ for all $j$. This will basically set $V_t(j) = max_iV_{t-1}(i)a_{ij}$, and allow you to interpolate the missing POS tag based on the transition probabilities alone.<br><br>
Note 2: In order to avoid additional problems with zero-probability transitions when applying your model to the test set, make sure that you tag the corpus sentence by sentence (i.e., compute the optimal tag sequence for each sentence independently). 
</div>

# 1. Exploring Tagged Corpus

In [18]:
CORPUS_train = "wsj_tagged_train.tt"
CORPUS_gold = "wsj_tagged_eval.tt"
CORPUS_test = "wsj_tagged_test.t"



def read_annotated(fname):
    '''Read the sentences from an annotated corpus.'''
    with open(fname, 'r') as f:
        sentences = [[]]
        for l in f.readlines():
            s = l.split()
            if s:
                # Next word, append to sentence.
                sentences[-1].append((s[0].lower(), s[1]))
            else:
                # Sentence has ended.
                sentences.append([])

        # Get rid of '[]' in the end.
        sentences.pop()
        return sentences


def read_unannotated(fname):
    '''Read sentences from an unannotated corpus.'''
    with open(fname, 'r') as f:
        sentences = [[]]
        for l in f.readlines():
            s = l.strip().lower()
            if s:
                # Next word, append to sentence.
                sentences[-1].append(s)
            else:
                # Sentence has ended.
                sentences.append([])

        # Get rid of '[]' in the end.
        sentences.pop()
        return sentences




In [19]:
# Splitting into train and test
train_set = read_annotated(CORPUS_train)
test_set = read_unannotated(CORPUS_test)

In [20]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]

In [21]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
tokens[:10]

['in', 'an', 'oct.', '19', 'review', 'of', '``', 'the', 'misanthrope', "''"]

In [22]:
# vocabulary
V = set(tokens)
# number of tags
T = set([pair[1] for pair in train_tagged_words])
print(T)

{'NN', 'VBD', 'LS', 'RBS', '.', 'NNP', 'RBR', '#', ':', 'RB', 'TO', 'NNS', '``', 'VBP', 'PRP', 'FW', 'VBG', 'WDT', 'CC', 'JJR', 'RP', 'WP', 'IN', 'VBN', 'PDT', 'EX', 'MD', "''", 'VB', 'POS', ')', 'VBZ', 'JJ', '$', 'JJS', 'NNPS', 'WP$', ',', '(', 'CD', 'UH', 'PRP$', 'WRB', 'DT', 'SYM'}


## 2. POS Tagging Algorithm - HMM

HMM algorithm is used in order to,assign the most probable tag to the word, given a sequence of words to be tagged, the task is to 
In general assign the tag that maximises the likelihood P(t/w). 
\
\
$P(t/w) = P(w/t). P(t) / P(w)$
\
\
where, 
\
\
P(w/t) is  the probability that given a tag, what is the probability of it being w. 

In [23]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

In [24]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

In [25]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [26]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

KeyboardInterrupt: 

In [ ]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [ ]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [ ]:

#alternative method for emissions probabilities
def calc_initial(sentences):
    d = defaultdict(lambda: 0)
    for s in sentences:
        # Count only the first tag in each sentence.
        tag = s[0][1]
        d[tag] += 1

    # Normalize.
    total = sum(d.values())
    return {k: v/total for k, v in d.items()}


def calc_transitions(sentences):
   
    bigrams = []
    for s in sentences:
        [bigrams.append((tag1, tag2)) for (_, tag1), (_, tag2) in zip(s, s[1:])]

    d = defaultdict(lambda: defaultdict(lambda: 0))
    for (b1, b2) in bigrams:
        d[b1][b2] += 1

    # Normalize.
    ret = {}
    for tag, dist in d.items():
        total = sum(dist.values())
        ret[tag] = {k: v/total for k, v in dist.items()}

    return ret


def calc_emissions(sentences):
    d = defaultdict(lambda: defaultdict(lambda: 0))
    for word, tag in chain.from_iterable(sentences):
        d[tag][word] += 1  # Note reversed ordering compared to the corpus!

    # Normalize.
    ret = {}
    for tag, dist in d.items():
        total = sum(dist.values())
        ret[tag] = {k: v/total for k, v in dist.items()}

    return ret


class Train:
   
    def __init__(self, fname=CORPUS_train):
        self.sen = read_annotated(fname)


    def get_initial(self):
        return calc_initial(self.sen)


    def get_transitions(self):
        return calc_transitions(self.sen)


    def get_emissions(self):
        return calc_emissions(self.sen)

if __name__ == "__main__":
    sen = read_annotated(CORPUS_train)
    print(calc_initial(sen))
    #print(calc_transitions(sen))
    #print(calc_vocabulary_size(sen))
    #print(calc_emissions(sen))
 

<div class="alert alert-block alert-info">
Explain which smoothing procedures you used and any observations.
</div>

*your text goes here*

## Problem 3: Evaluation [34 points]

<div class="alert alert-block alert-info">
Once you have trained a model, evaluate it on the unseen data from the test set. Run the Viterbi algorithm with each of your models, and output a tagged corpus in the two-column CoNLL format (*.tt). Use the provided evaluation script `tagging_eval.py`, which you can run on a gold annotated file and your own tagging results.<br><br>
Run it on the output of your tagger and the evaluation set and report your results. 
</div>

In [ ]:
# list of sents
test_run = [test_set[i] for i in len(test_set)]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
test_run

In [13]:
# your code goes here


if __name__ == '__main__':
    import sys
    import string
    
    try:
        fsock_gold=read_unannotated(CORPUS_gold)
        fsock_test=read_unannotated(CORPUS_test)
        sentences = 0
        s_right = 0
        words = 0
        w_right = 0
        for gline in fsock_gold:
             gsplitline=gline.rstrip().split()
             tline = fsock_test.readline() 
             if not tline: break
             tsplitline=tline.rstrip().split()
             glinelen = len(gsplitline)
             tlinelen = len(tsplitline)
             if not glinelen == tlinelen:
                 print ('Unmatched gold standard and test')
                 print ('gold: %s' % gline)
                 print ('test: %s' % tline)
                 break
             index=glinelen-1
             serror=0
             if glinelen>0:
                 while index > -1:
                     words = words + 1
                     gelem=gsplitline[index]
                     telem=tsplitline[index] 
                     if gelem == telem:
                         w_right = w_right +1
                     else: serror=1
                     index = index - 1
                 sentences = sentences + 1
                 if not serror: s_right = s_right + 1
             else: continue
        fsock_gold.close()
        fsock_test.close()
        print ('Computing scores')
        print ('%d right out of %d tags correct\n' % (w_right, words))
        print ('%.2f per cent word accuracy\n' % (float(w_right)/float(words) * 100.0, ))
        print ('%.2f per cent sentence accuracy\n' % (float(s_right)/float(sentences) * 100.0, ))
    except IOError:
        print ("Unable to open %s or %s" % (gold_file, test_file))

AttributeError: 'list' object has no attribute 'rstrip'

<div class="alert alert-block alert-info">
Discuss the results of the different tagger versions.
</div>

*your discussion goes here*

## Extra Credit [10 points]

<div class="alert alert-block alert-info">
The task is challenging as it stands. However, feel free to go further for extra credit, e.g. by doing one of the following: implement better unknown word handling, use a trigram tagger, plot a learning curve for your tagger (accuracy as a function of training data size), plot a speed vs. sentence length curve.
</div>

In [ ]:
# learning curve of tagger
import matplotlib.pyplot as plt

plt.style.use('seaborn')
plt.plot(train_sizes, train_F1_mean, label = 'Training error')
plt.plot(train_sizes, validation_F1_mean, label = 'Validation error')
plt.ylabel('Score', fontsize = 14)
plt.xlabel('Training set size', fontsize = 14)
plt.title('Learning curves for a linear regression model', fontsize = 18, y = 1.03)
plt.legend()
plt.ylim(0,40)

*your discussion*